In [ ]:
from datasets import load_dataset, Dataset, concatenate_datasets

In [ ]:
train_essays = load_dataset('csv', data_files = '/kaggle/input/llm-detect-ai-generated-text/train_essays.csv')
test_essays = load_dataset('csv', data_files = '/kaggle/input/llm-detect-ai-generated-text/test_essays.csv')
train_prompts = load_dataset('csv', data_files = '/kaggle/input/llm-detect-ai-generated-text/train_prompts.csv')

In [ ]:
train_essays, test_essays, train_prompts

In [ ]:
train_test_split = train_essays['train'].train_test_split(test_size = 0.2)

In [ ]:
train = train_test_split['train'].rename_column('generated' , 'label')
val = train_test_split['test'].rename_column('generated' , 'label')
test = test_essays['train']

In [ ]:
train , val, test

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

model_ckpt = '/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1'

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_use_double_quant = True,
    bnb_4bit_quant_type = 'nf4',
    bnb_4bit_compute_dtype = torch.bfloat16
)

def get_new_model():
    return AutoModelForCausalLM.from_pretrained(model_ckpt, quantization_config = bnb_config, low_cpu_mem_usage = True)

In [ ]:
prompts = ['<prompt> ' + ' '.join(text.split()[:-7]) for text in train_prompts['train']['instructions']]

In [ ]:
texts = [' '.join(text.split()[:10]) for text in train['text']]

In [ ]:
def make_prompts(prompts, train_data):
    return  [prompts[prompt_id] + ' <response> ' + text  for prompt_id, text in zip(train_data['prompt_id'], texts)]
      
    
input_texts = Dataset.from_dict({'text' : make_prompts(prompts, train)})

In [ ]:
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'left'

def preprocess(batch):
    return tokenizer(batch['text'], padding = 'max_length', truncation = True, return_tensors = 'pt', max_length = 128) 

inputs = input_texts.map(preprocess, batched = True, batch_size = 16)

In [ ]:
import gc

gc.collect()
torch.cuda.empty_cache()

In [ ]:
model = get_new_model()

In [ ]:
import gc

gc.collect()
torch.cuda.empty_cache()

In [ ]:
%%time

generated_texts = []
batch_size = 32

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
for i in range(0, len(inputs), batch_size):
    start = i; end = min(i + batch_size, len(inputs))
    batch = inputs[start:end]
    
    input_ids =  torch.tensor(batch['input_ids']).to(device)
    attention_mask =  torch.tensor(batch['attention_mask']).to(device)
     
    with torch.no_grad():
        outputs = model.generate(
            input_ids = input_ids,
            attention_mask = attention_mask,
            max_new_tokens = 1000,
            top_p = 0.9,
            top_k = 50,
            temperature = 0.7,
            repetition_penalty = 1.2,
            do_sample = True
        )
    generated_texts.extend([tokenizer.decode(output, skip_sepcial_tokens = True) for output in outputs])
    
    print(f'batch {i // batch_size + 1} generation completed')
    break

In [ ]:
cleaned_texts = [text.replace(tokenizer.pad_token, '').split('<response>')[1].strip() for text in generated_texts]

In [ ]:
gen_data = Dataset.from_dict({'text' : cleaned_texts, 'label' : [1 for _ in range(len(cleaned_texts))]})
train_ = train.remove_columns(['id','prompt_id'])
train_data = concatenate_datasets([train_, gen_data])

In [ ]:
torch.cuda.empty_cache()

In [ ]:
from transformers import Gemma2ForSequenceClassification, GemmaTokenizerFast
import torch

gemma_model_ckpt = '/kaggle/input/gemma-2/transformers/gemma-2-2b/1'
gemma_tokenizer = GemmaTokenizerFast.from_pretrained(gemma_model_ckpt)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_use_double_quant = True,
    bnb_4bit_quant_type = 'nf4',
    bnb_4bit_compute_dtype = torch.bfloat16
)

def get_classifier():
    return Gemma2ForSequenceClassification.from_pretrained(gemma_model_ckpt, num_labels = 2, quantization_config = bnb_config, low_cpu_mem_usage = True)

In [ ]:
def tokenize_data(batch):
    return gemma_tokenizer(batch['text'], padding = True, truncation = True, max_length = 512)

train_ds = train_data.map(tokenize_data, batched = True)
val_ds = val.map(tokenize_data, batched = True)
test_ds = test.map(tokenize_data, batched = True)

In [ ]:
train_ds, val_ds, test_ds

In [ ]:
import os

os.environ['WANDB_DISABLED'] = 'true'

In [ ]:
from transformers import TrainingArguments, Trainer

In [ ]:
from sklearn.metrics import roc_auc_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds)
    return {'f1' : f1}

In [ ]:
per_device_bs =  2
output_dir = '/kaggle/working/gemma2'

args = TrainingArguments(
    output_dir = output_dir,
    num_train_epochs = 2,
    learning_rate = 1e-5,
    per_device_train_batch_size = per_device_bs,
    per_device_eval_batch_size = per_device_bs,
    eval_strategy = 'epoch',
    save_strategy = 'epoch',
    logging_steps = 100,
    load_best_model_at_end = True,
    overwrite_output_dir = True,
)

In [ ]:
import gc

gc.collect()
torch.cuda.empty_cache()

In [ ]:
classifier = get_classifier()

In [ ]:
from peft import LoraConfig, get_peft_model


lora_config = LoraConfig(
    r = 8,
    lora_alpha = 16,
    target_modules = ['q_proj','v_proj','k_proj', 'o_proj'],
    bias = 'none',
    task_type = 'SEQ_CLS',
    init_lora_weights = True,
)

peft_model =  get_peft_model(classifier, lora_config)

In [ ]:
trainer = Trainer(
    model = peft_model,
    args = args,
    train_dataset = train_ds,
    eval_dataset = val_ds,
    compute_metrics = compute_metrics,
    tokenizer = gemma_tokenizer
)

trainer.train()

In [ ]:
preds = trainer.predict(test_ds)

preds

In [ ]:
x = torch.sigmoid(torch.tensor(preds.predictions))

x

In [ ]:
import pandas as pd

sub = pd.DataFrame({
    'id' : test_['train']['id'],
    'generated' : x[:, 0]
})

sub.to_csv('submission.csv', index = False)

In [ ]:
import shutil 


shutil.rmtree('/kaggle/working/bert')